In [3]:
import ast
import string

In [127]:
def adds(a, b):
    return f"[{a},{b}]"
    
def reduces(a):
    # print("Reducing:", a)
    while True:
        a, r = explodes(a)
        if r: pass # print("Exploded:", a)
        else:
            a, r = splits(a)
            if r: pass # print("Reduced:", a)
            else: break
    return a


In [128]:
def splits(a):
    tosplit = a
    cursor = 0
    while cursor < len(tosplit):
        if tosplit[cursor] in string.digits and tosplit[cursor + 1] in string.digits:
            n = int(tosplit[cursor: cursor + 2])
            l,r = n // 2, n // 2 + n % 2
            tosplit = tosplit[:cursor] + str([l, r]).replace(" ", "") + tosplit[cursor + 2:]
            return tosplit, True
        cursor += 1
    return tosplit, False

In [129]:
def explodes(a : str):
    nested = a
    cursor = 0
    depth = 0

    def findFirstClosingBracketFrom(start: int):
        return nested[start:].find("]") + start
    def getPairToExplode(start: int, end: int):
            pair = nested[start + 1:end].split(",")
            return int(pair[0]), int(pair[1])
    def getFirstLeftNumberBounds(_from: int):
        lc = _from
        while lc > 1:
            if nested[lc] in string.digits:
                if nested[lc - 1] in string.digits:
                    return lc - 1, lc + 1
                return lc, lc + 1
            lc -= 1
        return -1, -1
    def getFirstRightNumberBounds(_from : int):
        rc = _from
        while rc < len(nested) - 1:
            if nested[rc] in string.digits:
                if nested[rc + 1] in string.digits:
                    return rc, rc + 2
                return rc, rc + 1
            rc += 1
        return -1, -1
            

    while cursor < len(nested):
        c = nested[cursor]
        if c == "[": depth += 1
        elif c == "]": depth -= 1

        if depth == 5:
            firstClosingBracket = findFirstClosingBracketFrom(cursor)
            l, r = getPairToExplode(cursor, firstClosingBracket)
            lmin, lmax = getFirstLeftNumberBounds(cursor)
            rmin, rmax = getFirstRightNumberBounds(firstClosingBracket)
            if rmin != -1:
                rn = int(nested[rmin:rmax]) + r
                nested = nested[:rmin] + str(rn) + nested[rmax:]
            nested = nested[:cursor] + '0' + nested[firstClosingBracket + 1:]
            if lmin != -1:
                ln = int(nested[lmin:lmax]) + l
                nested = nested[:lmin] + str(ln) + nested[lmax:]
            return nested, True
        cursor+=1
    return nested, False


In [130]:
a = explodes("[[[[[9,8],1],2],3],4]") # right only
b = explodes("[7,[6,[5,[4,[3,2]]]]]") # left only
c = explodes("[[6,[5,[4,[3,2]]]],1]") # both ways
d = explodes("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]") # [3,2] unaffected
e = explodes("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]")
a[0], b[0], c[0], d[0], e[0]

('[[[[0,9],2],3],4]',
 '[7,[6,[5,[7,0]]]]',
 '[[6,[5,[7,0]]],3]',
 '[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]',
 '[[3,[2,[8,0]]],[9,[5,[7,0]]]]')

In [131]:
x, y = "[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]"
z = adds(x, y)
res = "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]".replace(" ", "")

In [132]:
reduces(z), res == reduces(z)

('[[[[0,7],4],[[7,8],[6,0]]],[8,1]]', True)

In [133]:
nums = open("puzzle18_es.txt").read().split("\n")
r = nums[0]
for x in range(1, len(nums)):
    r = adds(r, nums[x])
    r = reduces(r)
print(r)

[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]


In [140]:
# https://www.reddit.com/r/adventofcode/comments/rj7cgm/comment/hp2fmmn/?utm_source=share&utm_medium=web2x&context=3
def magnitudes(a):
    l = ast.literal_eval(a)
    return magnitude(l)

def magnitude(a):
    if type(a) == int: return a
    else: return 3 * magnitude(a[0]) + 2 * magnitude(a[1])




In [136]:
magnitudes("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]")

3488

In [138]:
nums = open("puzzle18.txt").read().split("\n")
res = nums[0]
for i in range(1, len(nums)):
    res = adds(res, nums[i])
    res = reduces(res)
mag = magnitudes(res)
print("Reduced sum result is:", res)
print("Magnitude:", mag)

Reduced sum result is: [[[[6,6],[0,6]],[[6,6],[7,7]]],[[[7,7],[7,7]],[[7,8],[8,9]]]]
Magnitude: 3816


In [139]:
sumMags = []
nums = open("puzzle18.txt").read().split("\n")
for i1 in range(len(nums)):
    for i2 in range(i1, len(nums)):
        sum1 = adds(nums[i1], nums[i2])
        sum2 = adds(nums[i2], nums[i1])
        red1 = reduces(sum1)
        red2 = reduces(sum2)
        sumMags += [magnitudes(red1), magnitudes(red2)]
print("Maximum magnitude: ", max(sumMags))

Maximum magnitude:  4819
